In [ ]:
!pip install -q pyod

In [ ]:
from stable_baselines3.common.distributions import CategoricalDistribution, SelfCategoricalDistribution
# from stable_baselines3.common.type_aliases import PyTorchObs
from typing import Optional

# from generator import generate_dataset
# from env import Env_FS
import torch
from sklearn.model_selection import train_test_split
import numpy as np
# from generator import create_ODDS
import random
import os
# from generator import generate_dataset
# import env
from sklearn.model_selection import train_test_split
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import NearestNeighbors
from stable_baselines3.common.policies import ActorCriticPolicy
from torch.distributions import Distribution, Categorical
from functools import partial
import gymnasium as gym
import numpy as np
from gymnasium import spaces
import random
from sklearn.neighbors import NearestNeighbors
import time

In [ ]:
class Env_FS(gym.Env):

    def __init__(self, X, y, o_b, i_b, rho = 0.1, model = None, max_steps = None, gamma = 0.9, max_neg_iter = 3):

        super(Env_FS, self).__init__()

        self.gamma= gamma
        self.max_neg_iter = max_neg_iter

        self.X = X
        self.y = y
        self.rho = rho
        self.n_features = X.shape[1]
        if max_steps is None:
            self.max_steps = min(self.n_features, 30)
        else:
            self.max_steps = max_steps
        self.o_b = o_b
        self.i_b = i_b
        self.model = model

        self.action_space = spaces.Discrete(self.n_features)
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.n_features,), dtype=np.int8)

        self.state = None
        self.visited = set()
        self.selected = None

        self.AOR = np.zeros((2, self.n_features))
        self.MDR = np.zeros((2, self.n_features))

        self.call_to_step = 0
        self.episode_number = 0
        self.neg_reward_count = 0
        self.time_step = 0

    def seed(self, seed=None):
        np.random.seed(seed)
        return [seed]

    def step(self, action):
        self.call_to_step = self.call_to_step + 1

        action = int(action)

        if self.state[action] == 1:
            return self.state.copy(), -10.0, False, False, {}

        obs_tensor = torch.tensor(self.state, dtype=torch.float32).unsqueeze(0).to(self.model.device)

        with torch.no_grad():
            value_t = self.model.policy.predict_values(obs_tensor).cpu().item()

        if len(self.selected)!=0:
            score_cur = self.evaluation_subspace(self.selected, rho=self.rho)
        else:
            score_cur = 0
        #state_prec = self.state.copy()
        self.selected.append(action)
        self.state[action] = 1 

        score_suc = self.evaluation_subspace_wo_rho(self.selected, rho=self.rho)

        reward = score_suc - score_cur

        obs_tensor = torch.tensor(self.state, dtype=torch.float32).unsqueeze(0).to(self.model.device)

        with torch.no_grad():
            value_t1 = self.model.policy.predict_values(obs_tensor).cpu().item()
        
        if not hasattr(self, 'time_step'):
            self.time_step = 0
        
        #if self.call_to_step >= 5000:
        self.AOR[0, action] += 1
        self.MDR[0, action] += 1
        #######
        #AOR
        #
        self.AOR[1, action] = self.AOR[1, action] + (value_t - value_t1)
        #
        #
        
        #MDR
        #
        self.MDR[1, action] = self.MDR[1, action] + (self.gamma**self.time_step * reward)
        #
        #
        
        #####
        # self.AOR[1, action] = self.AOR[1, action] + (reward)


        self.time_step += 1

        if reward<0:
            self.neg_reward_count +=1
        else:              
            self.neg_reward_count = 0
        
        #if self.call_to_step > 5000 and self.call_to_step % 1000 == 0:
        if self.call_to_step % 1000 == 0:
            print(f'{self.call_to_step:.0f}:')
            for f in range(1,self.n_features):
                aor_count = self.AOR[0,f]
                aor_value = self.AOR[1,f]
                mdr_count = self.MDR[0,f]
                mdr_value = self.MDR[1,f]
                if aor_count>0:
                    aor_value /= aor_count
                if mdr_count>0:
                    mdr_value /= mdr_count
                #print(f'[{f:.0f}:{aor_count:.0f}:{aor_value:.3f}]', end='')
            if self.call_to_step > 500:
                print(f'mean episode length = {self.call_to_step/self.episode_number:.1f}\n')


        
        # print(f"\rF[0] = {self.AOR[0,0]:5f} - {self.AOR[1,0]:.5f}", end="")
        # s="\033[F"
        # for f in range(1,self.n_features):
        #     print(f"\nF[{f}] = {self.AOR[0,f]:5f} - {self.AOR[1,f]:.5f}", end="")
        #     s = s + "\033[F"
        # print(f"{s}", end=""))
            
        
        done = len(self.selected) >= self.max_steps

        done = done or self.neg_reward_count >= self.max_neg_iter

        if done:
            self.episode_number += 1
        
        # print(f"Step {len(self.selected)} - Action {action}, Reward {reward:.4f}, V(s): {value_t:.4f} → V(s’): {value_t1:.4f}")

        return self.state.copy(), reward, done, done, {}
        #return self.state.copy(), reward, done, {}

    

    def reset(self, seed=None):
        self.neg_reward_counter = 0
        self.time_step = 0
        self.selected = []
        self.state = np.zeros(self.n_features, dtype=np.int8)
        return self.state, {}

    def outlier_scores(self, features, k=50):
        features = np.array(features, dtype=int)

        X_fit = self.X[self.y == 0][:, features]
        if X_fit.ndim == 1:
            X_fit = X_fit.reshape(-1, 1)

        nbrs = NearestNeighbors(n_neighbors=k + 1, algorithm='auto').fit(X_fit)

        X_ob = self.X[self.o_b][:, features]
        if X_ob.ndim == 1:
            X_ob = X_ob.reshape(-1, 1)

        X_ib = self.X[self.i_b][:, features]
        if X_ib.ndim == 1:
            X_ib = X_ib.reshape(-1, 1)
        
        distances_o, _ = nbrs.kneighbors(X_ob)
        distances_i, _ = nbrs.kneighbors(X_ib)

        mean_distances_o = distances_o[:, 1:].mean(axis=1)
        mean_distances_i = distances_i[:, 1:].mean(axis=1)

        mean_distances_o = mean_distances_o/np.sqrt(features.shape[0])
        mean_distances_i = mean_distances_i/np.sqrt(features.shape[0])

        return mean_distances_o, mean_distances_i


    def evaluation_subspace(self, features, rho):

        rho_frac = rho * self.o_b.shape[0]
        rho_frac = int(rho_frac)

        o_os,i_os  = self.outlier_scores(features=features)

        o_os_sorted = np.argsort(o_os)
        rho_ids = o_os_sorted[:rho_frac+1]

        sc_o_b = np.mean(o_os[rho_ids])
        sc_i = np.mean(i_os)

        max_i_os = np.max(i_os)
        min_o_os = o_os[o_os_sorted[rho_frac]]

        ss = 0
        if (sc_o_b > sc_i) and (min_o_os > max_i_os):
            ss = np.mean(o_os) - sc_i

        return ss

    def evaluation_subspace_wo_rho(self, features, rho):

        o_os,i_os  = self.outlier_scores(features=features)

        o_os_sorted = np.argsort(o_os)
        sc_i = np.mean(i_os)

        max_i_os = np.max(i_os)

        ss = np.mean(o_os) - sc_i

        return ss

    def get_aor_scores(self):
        self.MDR[0, self.MDR[0,:]==0] = 1
        self.MDR[1, :]=self.MDR[1,:]/self.MDR[0,:]
        self.AOR[0, self.AOR[0,:]==0] = 1
        self.AOR[1, :]=self.AOR[1,:]/self.AOR[0,:]
        ranking_AOR = np.argsort(self.AOR[1, :])[::-1]
        ranking_MDR = np.argsort(self.MDR[1,:])[::-1]
        return ranking_AOR, self.AOR[1, ranking_AOR], ranking_MDR, self.MDR[1, ranking_MDR]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from scipy.io import loadmat
import os
from pyod.utils.utility import standardizer
import numpy.random

def generate_dataset(n_inliers=100, n_outliers=20, n_features=15, n_informative=7, radius = 5, seed=42):
    np.random.seed(seed)

    n_noise = n_features - n_informative

    #Inliers
    informative_inliers = np.random.normal(loc=0.0, scale=1.0, size=(n_inliers, n_informative))
    noise_inliers = np.random.normal(loc=0.0, scale=1.0, size=(n_inliers, n_noise))
    X_inliers = np.hstack([informative_inliers, noise_inliers])
    y_inliers = np.zeros(n_inliers)

    #Outliers
    '''informative_outliers = np.random.normal(loc=3.0, scale=7.0, size=(n_outliers, n_informative))
    noise_outliers = np.random.normal(loc=0.0, scale=1.0, size=(n_outliers, n_noise))
    X_outliers = np.hstack([informative_outliers, noise_outliers])
    y_outliers = np.ones(n_outliers)'''

    informative_outliers = []
    while len(informative_outliers) < n_outliers:
        candidate = np.random.uniform(low=-5, high=5, size=n_informative)
        if np.linalg.norm(candidate) > radius:
            informative_outliers.append(candidate)
    informative_outliers = np.array(informative_outliers)

    noise_outliers = np.random.normal(loc=0.0, scale=1.0, size=(n_outliers, n_noise))
    X_outliers = np.hstack([informative_outliers, noise_outliers])
    y_outliers = np.ones(n_outliers)

    X = np.vstack([X_inliers, X_outliers])
    y = np.concatenate([y_inliers, y_outliers])
    X, y = shuffle(X, y, random_state=seed)

    true_feature_importance = np.zeros(n_features)
    true_feature_importance[:n_informative] = 1

    perm = np.random.permutation(X.shape[1])

    X = X[:, perm]
    true_feature_importance = true_feature_importance[perm]

    return X, y, true_feature_importance.astype(int)



def create_ODDS(dataset, seed):
    np.random.seed(seed)
    mat = loadmat(os.path.join('/kaggle/input/train-sets/trainsets', dataset))

    X = mat['X']
    y = mat['y'].ravel()
    X = standardizer(X)

    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    ids_anomalies = np.where(y == 1)[0]

    return X, y, ids_anomalies

In [ ]:

class OutDistribution(CategoricalDistribution):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def proba_distribution(self: SelfCategoricalDistribution, action_logits: torch.Tensor, obs) -> SelfCategoricalDistribution:

        
        #print(f'action_logits in [{torch.min(action_logits)},{torch.max(action_logits)}]')
        #print(f'action_logits = {action_logits} --- obs = {obs}')

        #print(action_logits.logsumexp(dim=-1, keepdim=True))
        
        action_logits[obs == 1] = -float('inf')        

        #print(action_logits.logsumexp(dim=-1, keepdim=True))

        
        self.distribution = Categorical(logits=action_logits)

        #print(f'probs = {self.distribution.probs} -- obs = {obs}')

        
        return self


class OutPolicy(ActorCriticPolicy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        dist_kwargs = self.dist_kwargs
        if dist_kwargs is None:
            dist_kwargs = {}
        self.action_dist = OutDistribution(int(self.action_space.n), **dist_kwargs)


    
    def _get_action_dist_from_latent(self, latent_pi: torch.Tensor) -> Distribution:
        raise ValueError("Perché mi hai chiamato?")


    
    def _get_action_dist_from_latent(self, obs, latent_pi: torch.Tensor) -> Distribution:
        """
        Retrieve action distribution given the latent codes.
        :param latent_pi: Latent code for the actor
        :return: Action distribution
        """
        mean_actions = self.action_net(latent_pi)

        #########print(f'mean_actions = {mean_actions}')

        return self.action_dist.proba_distribution(action_logits=mean_actions, obs=obs)


    def my_get_actions(self, distribution, obs):
        
        probs = distribution.distribution.probs[0].cpu().clone()
        #probs = torch.exp(distribution.distribution.logits)
        
        probs = probs / torch.sum(probs)
        
        legal_actions = torch.where(obs[0].cpu() != 1)[0]
        
        cumprobs = torch.cumsum(probs[legal_actions], dim = 0)
        rnd = np.random.rand()
        
        pos = torch.searchsorted(cumprobs, rnd, right = True) - 1
        #pos = bisect.bisect_right(cumprobs, rnd) - 1
        
        action = torch.tensor(legal_actions[pos]).clone().detach().to('cuda')
        #action = torch.tensor(legal_actions[pos].clone().detach().to('cuda'))
        log_prob = torch.exp(distribution.distribution.logits[0][action]).detach().to('cuda')
        return action, log_prob
    

    
    def forward(self, obs: torch.Tensor, deterministic: bool = False) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Forward pass in all the networks (actor and critic)

        :param obs: Observation
        :param deterministic: Whether to sample or use deterministic actions
        :return: action, value and log probability of the action
        """
        # Preprocess the observation if needed
        features = self.extract_features(obs)

        if self.share_features_extractor:
            latent_pi, latent_vf = self.mlp_extractor(features)
        else:
            pi_features, vf_features = features
            latent_pi = self.mlp_extractor.forward_actor(pi_features)
            latent_vf = self.mlp_extractor.forward_critic(vf_features)
        # Evaluate the values for the given observations
        values = self.value_net(latent_vf)
        distribution = self._get_action_dist_from_latent(obs,latent_pi)
        actions = distribution.get_actions(deterministic=deterministic)
        log_prob = distribution.log_prob(actions)
        
        actions = actions.reshape((-1, *self.action_space.shape))  # type: ignore[misc]
        return actions, values, log_prob


    def q_function(self, obs: torch.Tensor):
        features = self.extract_features(obs)
        if self.share_features_extractor:
            latent_pi, latent_vf = self.mlp_extractor(features)
        else:
            pi_features, vf_features = features
            latent_pi = self.mlp_extractor.forward_actor(pi_features)
            latent_vf = self.mlp_extractor.forward_critic(vf_features)
        values = self.value_net(latent_vf)
        
        mean_actions = self.action_net(latent_pi)
        distribution = Categorical(logits=mean_actions)
        
        return distribution.probs




    
    def evaluate_actions(self, obs: torch.Tensor, actions: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, Optional[torch.Tensor]]:
        """
        Evaluate actions according to the current policy,
        given the observations.

        :param obs: Observation
        :param actions: Actions
        :return: estimated value, log likelihood of taking those actions
            and entropy of the action distribution.
        """
        # Preprocess the observation if needed
        features = self.extract_features(obs)
        if self.share_features_extractor:
            latent_pi, latent_vf = self.mlp_extractor(features)
        else:
            pi_features, vf_features = features
            latent_pi = self.mlp_extractor.forward_actor(pi_features)
            latent_vf = self.mlp_extractor.forward_critic(vf_features)
        distribution = self._get_action_dist_from_latent(obs, latent_pi)
        log_prob = distribution.log_prob(actions)
        values = self.value_net(latent_vf)
        entropy = distribution.entropy()
        return values, log_prob, entropy

    def get_distribution(self, obs: torch.Tensor) -> Distribution:
        """
        Get the current policy distribution given the observations.

        :param obs:
        :return: the action distribution.
        """
        features = super().extract_features(obs, self.pi_features_extractor)
        latent_pi = self.mlp_extractor.forward_actor(features)
        return self._get_action_dist_from_latent(obs, latent_pi)

In [ ]:
def test_outlier_scores(X_train,y_train,X_test,y_test,features,k=50):
    
    features = np.array(features, dtype=int)
    X_tr = X_train[y_train==0][:, features]
    if X_tr.ndim == 1:
            X_tr = X_tr.reshape(-1, 1)
    nbrs = NearestNeighbors(n_neighbors=k + 1, algorithm='auto').fit(X_tr)

    X_te = X_test[:, features]
    if X_te.ndim == 1:
            X_te = X_te.reshape(-1, 1)
    distances, _ = nbrs.kneighbors(X_te)

    mean_distances = distances.mean(axis=1)
    mean_distances = mean_distances/np.sqrt(features.shape[0])
        
    return mean_distances

In [ ]:
def evaluate_ss(X_train,y_train,features, o_b):

    X_ob = X_train[o_b]
    y_ob = y_train[o_b]
    scores = test_outlier_scores(X_train,y_train,X_ob,y_ob,features)
    sc_o_b = np.mean(scores)

    return sc_o_b

In [ ]:
def compute_ranking_scores(model, X_train, y_train, X_test, y_test, ranking, o_b = None, score_types=["q_auc", "v_aor"], rho=0.1, k=50):
    
    d = min(5000, len(ranking))
    #d = len(ranking)
    obs = np.zeros((1, d))
    obs_tensor = torch.tensor(obs.copy()).to('cuda')

    results = {}

    if 'v_aor' in score_types or 'q_aor' in score_types:
        V_f = np.zeros(d)
        Q_f = np.zeros((d, d))

        for i in range(d):
            ids = ranking[:i+1]
            obs[0] = 0
            obs[0, ids] = 1
            obs_tensor = torch.tensor(obs.copy()).to('cuda')

            V_f[i] = model.policy.predict_values(obs_tensor).item()

            if i < d - 1:
                Q_f[i, :] = model.policy.q_function(obs_tensor)[0].detach().cpu().numpy()

        if 'v_aor' in score_types:
            results['v_aor'] = V_f
        if 'q_aor' in score_types:
            results['q_aor'] = Q_f

    if 'auc' in score_types:
        auc_curve = np.zeros(d)
        for i in range(d):
            selected = ranking[:i+1]
            scores_out = test_outlier_scores(X_train, y_train, X_test, y_test, selected, k=k)
            auc_curve[i] = roc_auc_score(y_test, scores_out)
        results['auc'] = auc_curve

    if "q_ranking" in score_types or "q_auc" in score_types:
        
        greedy_ranking = np.zeros(d, dtype=np.int32)
        obs = np.zeros((1, d))
        obs_tensor = torch.tensor(obs.copy()).to('cuda')
        probs = model.policy.q_function(obs_tensor)[0].detach().cpu().numpy()
        greedy_ranking[0] = np.argmax(probs)

        for i in range(d - 1):
            obs[0] = 0
            ids = greedy_ranking[:i+1]
            obs[0, ids] = 1
            obs_tensor = torch.tensor(obs.copy()).to('cuda')
            probs = model.policy.q_function(obs_tensor)[0].detach().cpu().numpy()
            probs[obs[0] == 1] = -1
            greedy_ranking[i+1] = np.argmax(probs)

        results['q_ranking'] = greedy_ranking

        if "q_auc" in score_types:
            auc_greedy = np.zeros(d)
            for i in range(d):
                selected = greedy_ranking[:i+1]
                scores_out = test_outlier_scores(X_train, y_train, X_test, y_test, selected, k=k)
                auc_greedy[i] = roc_auc_score(y_test, scores_out)
            results['q_auc'] = auc_greedy

    if "subspace" in score_types:
        ss_s = np.zeros(d)
        for i in range(d):
            ss = evaluate_ss(X_train,y_train,ranking[:i+1], o_b)
            ss_s[i] = ss

        results['subspace'] = ss_s

    return results


In [ ]:
def plot_fs_analysis(aor_scores_train, auc_train, V_f, V_auc, aor_count_train, ss_s, r, steps, dataset_name=""):
    d = len(aor_scores_train)
    pos = np.arange(d)

    # --- AOR plot ---
    plt.figure()
    plt.plot(pos, aor_scores_train)
    plt.grid()
    plt.xlabel('Feature ranking')
    plt.ylabel('AOR')
    plt.title(f'Average Of Rewards {dataset_name}')
    plt.savefig(f'AOR_{dataset_name}_{r}_{steps}.png')

    # --- AUC Train plot ---
    auc = auc_train
    a_dim = np.argmax(auc)
    a_auc = auc[a_dim]
    a_dim += 1
    alpha0 = 0.5
    alpha = alpha0 / d
    quality = auc - alpha * pos
    q_dim = np.argmax(quality)
    q_auc = auc[q_dim]
    q_dim += 1
    ss_dim = np.argmax(ss_s)

    plt.figure()
    plt.axis([0, d, np.min(auc), np.max(auc) * 1.025])
    plt.plot(pos, auc, 'b-', label=f'AUC ({a_dim},{a_auc:.3f})')
    plt.plot(pos, quality, 'r--', label=f'Quality ({q_dim},{q_auc:.3f})')
    plt.plot(pos, V_auc, 'g-', label='AUC_V')
    plt.plot(q_dim-1, q_auc, 'ro', label="Best Quality")
    plt.plot(a_dim-1, a_auc, 'b^', label="Best AUC")
    plt.legend(loc='lower right')
    plt.grid()
    plt.xlabel('Feature ranking')
    plt.title(f'AUC & Quality: Training set {dataset_name}')

    print(f'[Train] Best AUC at dim={a_dim} ({a_dim/d*100:.2f}%), AUC = {a_auc:.5f}')
    print(f'[Train] Best Quality at dim={q_dim} ({q_dim/d*100:.2f}%), AUC = {q_auc:.5f}')
    plt.savefig(f'quality_train_{dataset_name}_{r}_{steps}.png')
    # --- AUC Test plot ---
    # auc = auc_test
    # a_dim = np.argmax(auc)
    # a_auc = auc[a_dim]
    # a_dim += 1
    # quality = auc - alpha * pos
    # q_dim = np.argmax(quality)
    # q_auc = auc[q_dim]
    # q_dim += 1

    # plt.figure()
    # plt.axis([0, d, np.min(auc), np.max(auc) * 1.025])
    # plt.plot(pos, auc, 'b-', label=f'AUC ({a_dim},{a_auc:.3f})')
    # plt.plot(pos, quality, 'r--', label=f'Quality ({q_dim},{q_auc:.3f})')
    # plt.plot(pos, V_auc, 'g-', label='AUC_V')
    # plt.plot(q_dim-1, q_auc, 'ro', label="Best Quality")
    # plt.plot(a_dim-1, a_auc, 'b^', label="Best AUC")
    # plt.legend(loc='lower right')
    # plt.grid()
    # plt.xlabel('Feature ranking')
    # plt.title(f'AUC & Quality: Test set {dataset_name}')

    # print(f'[Test] Best AUC at dim={a_dim} ({a_dim/d*100:.2f}%), AUC = {a_auc:.5f}')
    # print(f'[Test] Best Quality at dim={q_dim} ({q_dim/d*100:.2f}%), AUC = {q_auc:.5f}')
    # plt.savefig(f'quality_test_{dataset_name}_{r}_{steps}.png')
    
    # --- AOR count plot ---
    plt.figure()
    plt.plot(pos, aor_count_train)
    plt.xlabel('Feature ranking')
    plt.title(f'Feature count {dataset_name}')
    plt.grid()
    plt.savefig(f'AOR_count_{dataset_name}_{r}_{steps}.png')
    
    # --- Value function AOR plot ---
    plt.figure()
    plt.plot(pos, V_f)
    plt.title(f'Value Function {dataset_name}')
    plt.xlabel('Feature ranking')
    #plt.ylabel('')
    plt.grid()
    plt.savefig(f'Value_f_{dataset_name}_{r}_{steps}.png')

    # --- Subspace score plot ---
    plt.figure()
    plt.plot(pos, ss_s, label=f'SS ({ss_dim},{ss_s[ss_dim]:.3f})', color='m')
    plt.title(f'Subspace score {dataset_name}')
    plt.xlabel('Feature ranking')
    plt.grid()
    plt.legend(loc='lower right')
    plt.savefig(f'Subspace_score_{dataset_name}_{r}_{steps}.png')

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def make_env(rho=0.1, gamma=0.9, max_neg_iter=3):
    env = Env_FS(X_train, y_train, new_o_indices, new_i_indices, rho=rho, gamma=gamma, max_neg_iter=max_neg_iter)
    env_ref['env'] = env
    return env

In [ ]:
real_ds = True
high_dims = False

if high_dims:
    n_variant = 1
    datasets = ['GSE13576']
else:
    n_variant = 5
    datasets = ['arrhythmia',
            #'cardio',
            #'ionosphere',
            'letter',
            #'mnist',
            #'musk',
            'optdigits',
            #'pendigits',
            #'pima',
            # 'satellite',
            # 'satimage-2',
            #'shuttle',
            #'vertebral',
            #'vowels',
            # 'wbc'
               ]
    #datasets = ['InternetAds']
    datasets = ['Portmap']


'''datasets = ['arrhythmia.mat',
            #'cardio.mat',
            #'ionosphere.mat',
            #'letter.mat'
           ]'''


n_inliers = 500
n_outliers = int(n_inliers * 0.10)
n_features = 300
n_informative = 5

rho = 0.5

#seed = np.random.randint(0xffffffff)
seed = 2568117345
print(f"SEED Generale: {seed}")
set_seed(seed)

#
#
n_runs =1
#
#

time_steps = [50000]

gammas = [0.99]
# gamma = 0.95
max_neg_iter = 3

seeds = np.random.randint(0, 0xffffffff, len(datasets) * n_variant * n_runs)

#stats = []

for r in range(n_runs):
    for id_d in range(len(datasets)):
        for gamma in gammas:
            for steps in time_steps:
                for v in range(n_variant):
                    id_seed = r * (len(datasets) * n_variant) + id_d * n_variant + v
                    seed = int(seeds[id_seed])
                    print(f"\033[1m***SEED run {r} dataset {id_d} = {seed}***")
                    
                    set_seed(seed)
        
                    if real_ds:
                        if high_dims:
                            dataset = datasets[id_d]
                            X = np.load(f'/kaggle/input/internetads/{dataset}_X.npy')
                            y = np.load(f'/kaggle/input/internetads/{dataset}_y.npy')
    
                            o_indices_all = np.where(y == 1)[0]
                            i_indices = np.where(y == 0)[0]
                            if len(i_indices) > 10000:
                                i_indices = np.random.choice(i_indices, size=10000, replace=False)
                
                            o_indices = np.random.choice(o_indices_all, size=min(20, len(o_indices_all)), replace=False)
                
                            selected_indices = np.concatenate([o_indices, i_indices])
                
                            X_train = X[selected_indices]
                            y_train = y[selected_indices]
                            new_o_indices = np.where(y_train == 1)[0]
                            new_i_indices = np.where(y_train == 0)[0]
    
                            new_i_indices = np.random.choice(new_i_indices, size=min(20, len(new_i_indices)), replace=False)
        
                        else:
                            dataset = f'{datasets[id_d]}_{v}'
                            #X, y,_ = create_ODDS(dataset, seed = seed)
                            # data = np.load(f'/kaggle/input/train-sets-odds/{dataset}.npz')
                            data = np.load(f'/kaggle/input/portmap-03-11-v3/{dataset}.npz')
                            X_train = data['X']
                            y_train = data['y']
    
                            new_o_indices = np.where(y_train == 1)[0]
                            new_i_indices = np.where(y_train == 0)[0]
            
                            y_train[y_train == -1] = 0
                        
                        print(f"Using dataset {dataset}, dim: {X_train.shape[1]}, n_sample: {X_train.shape[0]}")
                    
                    else:
                        dataset = f"synthetic_{n_inliers}_{n_outliers}"
                        X, y, important_mask = generate_dataset(n_inliers=n_inliers, n_outliers=n_outliers, n_features=n_features, n_informative=n_informative,seed=seed)
                        print(f"Using dataset {dataset} with mask {np.where(important_mask==1)[0]}")
        
                        o_indices_all = np.where(y == 1)[0]
                        i_indices = np.where(y == 0)[0]
                        if len(i_indices) > 10000:
                            i_indices = np.random.choice(i_indices, size=10000, replace=False)
            
                        o_indices = np.random.choice(o_indices_all, size=min(20, len(o_indices_all)), replace=False)
            
                        selected_indices = np.concatenate([o_indices, i_indices])
            
                        X_train = X[selected_indices]
                        y_train = y[selected_indices]
                        new_o_indices = np.where(y_train == 1)[0]
                        new_i_indices = np.where(y_train == 0)[0]
                        new_i_indices = np.random.choice(new_i_indices, size=min(20, len(new_i_indices)), replace=False)
            
                    env_ref = {}
        
                    #policy = OutPolicy()
                    vec_env = make_vec_env(partial(make_env, rho=rho, gamma=gamma, max_neg_iter=max_neg_iter), n_envs=1)
                    model = PPO(OutPolicy, vec_env, verbose=0, device='cuda', gamma=gamma)
        
                    env_ref['env'].model = model
                    #start_time = time.time()
                    model.learn(total_timesteps=steps, progress_bar=False)
                    #end_time = time.time()
        
                    #duration = end_time - start_time
                    #stats.append({
                    #    'Dataset': dataset,
                    #    'Samples': X_train.shape[0],
                    #    'Features': X_train.shape[1],
                    #    'Timesteps': steps,
                    #    'Train Time (s)': duration
                    #})
        
                    #print(stats)
                    
                    aor_ranking_train, aor_scores_train, mdr_ranking_train,mdr_scores_train = env_ref['env'].get_aor_scores()
        
        
                    aor_count_train = env_ref['env'].AOR[0,:]
                    aor_count_train = aor_count_train[aor_ranking_train]

                    mdr_count_train = env_ref['env'].MDR[0,:]
                    mdr_count_train = mdr_count_train[mdr_ranking_train]
                    
        
                    model.save(f"dir/FSwRL_{dataset}_{steps}_{r}_{gamma}")
        
                    #out_ids = np.setdiff1d(o_indices_all, o_indices)
                    #ids_X_test = np.concatenate([i_indices, out_ids])
        
                    #X_test = X[ids_X_test]
                    #y_test = y[ids_X_test]
        
        
                    aor_auc_train = np.zeros(min(5000,len(aor_ranking_train)))
                    mdr_auc_train = np.zeros(min(5000,len(mdr_ranking_train)))
                    #auc_test = np.zeros(ranking_train.shape)
                    aor_subspace_scores = np.zeros(min(5000,len(aor_ranking_train)))
                    mdr_subspace_scores = np.zeros(min(5000,len(mdr_ranking_train)))
                    for i in range(min(5000,len(aor_ranking_train))):
                        #scores_out = test_outlier_scores(X_train, y_train, X_test, y_test, ranking_train[:i+1], k=50)
                        #auc_value = roc_auc_score(y_test, scores_out)
                        #auc_test[i] = auc_value
                        scores_out = test_outlier_scores(X_train, y_train, X_train, y_train, aor_ranking_train[:i+1], k=50)
                        auc_value = roc_auc_score(y_train, scores_out)
                        aor_auc_train[i] = auc_value
                        ss = evaluate_ss(X_train,y_train, aor_ranking_train[:i+1], new_o_indices)
                        aor_subspace_scores[i] = ss

                        scores_out = test_outlier_scores(X_train, y_train, X_train, y_train, mdr_ranking_train[:i+1], k=50)
                        auc_value = roc_auc_score(y_train, scores_out)
                        mdr_auc_train[i] = auc_value
                        ss = evaluate_ss(X_train,y_train,mdr_ranking_train[:i+1], new_o_indices)
                        mdr_subspace_scores[i] = ss
                        
                    print('---AUC e SS calcolati---')
                    results = compute_ranking_scores(model, X_train, y_train, X_train, y_train, aor_ranking_train, score_types = ['q_auc','v_aor'])
                    #value_func = compute_ranking_scores(model, X_train, y_train, X_train, y_train, ranking_train, score_type = 'v_aor')
                    print('---Altre misure calcolate---')
                    #----PLOT-----
                    # plot_fs_analysis(aor_scores_train, aor_auc_train, results['v_aor'], results['q_auc'], aor_count_train, aor_subspace_scores, r, steps, dataset_name=f"{dataset}")

                    # plot_fs_analysis(mdr_scores_train, mdr_auc_train, results['v_aor'], mdr_count_train, mdr_subspace_scores, r, steps, dataset_name=f"{dataset}")
                    #----Salvataggio risultati-----
                    
                    #np.save(f'auc_{dataset}_{steps}_{r}.npy', auc_test)
                    np.save(f'aor_auc_train_{dataset}_{steps}_{gamma}_{r}.npy', aor_auc_train)
                    np.save(f'mdr_auc_train_{dataset}_{steps}_{gamma}_{r}.npy', mdr_auc_train)
                    np.save(f'AOR_subspace_scores_{dataset}_{steps}_{gamma}_{r}.npy', aor_subspace_scores)
                    np.save(f'MDR_subspace_scores_{dataset}_{steps}_{gamma}_{r}.npy', mdr_subspace_scores)
                    np.save(f'q_ranking_{dataset}_{steps}_{gamma}_{r}.npy', results['q_ranking'])
                    np.save(f'q_auc_{dataset}_{steps}_{gamma}_{r}.npy', results['q_auc'])
                    np.save(f'v_function_AOR_{dataset}_{steps}_{gamma}_{r}.npy', results['v_aor'])            
                    np.save(f"AOR_ranking_{dataset}_{steps}_{gamma}_{r}.npy", aor_ranking_train)
                    np.save(f"AOR_{dataset}_{steps}_{gamma}_{r}.npy", aor_scores_train)
                    np.save(f"count_{dataset}_{steps}_{gamma}_{r}.npy", aor_count_train)
                    np.save(f"MDR_ranking_{dataset}_{steps}_{gamma}_{r}.npy", mdr_ranking_train)
                    np.save(f"MDR_{dataset}_{steps}_{gamma}_{r}.npy", mdr_scores_train)
                    np.save(f"MDR_count_{dataset}_{steps}_{gamma}_{r}.npy", mdr_count_train)
                    
                    #print(f"Max AUC ottenuto: {np.max(all_aucs):.4f} con {np.argmax(all_aucs)} feature")
                    print(f"Ranking: {aor_ranking_train}")
                    print(f"Ranking: {mdr_ranking_train}")

            

            
            

In [ ]:

# obs = np.zeros((1, ranking_train.shape[0]))

# V_f = np.zeros(ranking_train.shape[0])
# Q_f = np.zeros((ranking_train.shape[0],ranking_train.shape[0]))

# for i in range(ranking_train.shape[0]):
#     ids = ranking_train[:i+1]    
#     obs[0, ids] = 1
#     obs_tensor = torch.tensor(obs.copy()).to('cuda')
#     V_f[i] = model.policy.predict_values(obs_tensor)
#     if i < ranking_train.shape[0] - 1:
#         Q_f[i,:] = model.policy.q_function(obs_tensor)[0].detach().cpu().numpy()


# V_ranking_train = np.zeros(ranking_train.shape, dtype=np.int32)
# obs = np.zeros((1, ranking_train.shape[0]))
# probs = model.policy.q_function(obs_tensor)[0].detach().cpu().numpy()
# V_ranking_train[0] = np.argmax(probs)
# for i in range(len(V_ranking_train)-1):
#     ids = V_ranking_train[:i+1]    
#     obs[0, ids] = 1
#     obs_tensor = torch.tensor(obs.copy()).to('cuda')
#     probs = model.policy.q_function(obs_tensor)[0].detach().cpu().numpy()
#     #print(i,probs)
#     probs[obs[0] == 1] = -1
#     V_ranking_train[i+1] = np.argmax(probs)


# V_auc = np.zeros(V_ranking_train.shape)

# for i in range(len(V_ranking_train)):
#     scores_out = test_outlier_scores(X_train, y_train, X_test, y_test, V_ranking_train[:i+1], k=50)
#     auc_value = roc_auc_score(y_test, scores_out)
#     V_auc[i] = auc_value


In [ ]:
# ranking_train_tensor = torch.tensor(ranking_train.reshape((1,ranking_train.shape[0])).copy())
# ranking_train_tensor = ranking_train_tensor.to('cuda')

# obs = np.zeros((1, ranking_train.shape[0]))

# i = 1

# ss_s = np.zeros(ranking_train.shape[0])

# for i in range(ranking_train.shape[0]):
#     ss = evaluate_ss(X_train,y_train,X_test,y_test,ranking_train[:i+1], new_o_indices)
#     ss_s[i] = ss
#     #print(f'Score sottospazio di dim {i}: {ss}')
#     ids = ranking_train[:i+1]


arrhythmia.mat: (452, 274), n_an: 66
cardio.mat: (1831, 21), n_an: 176
glass.mat: (214, 9), n_an: 9
ionosphere.mat: (351, 33), n_an: 126
letter.mat: (1600, 32), n_an: 100
lympho.mat: (148, 18), n_an: 6
mnist.mat: (7603, 100), n_an: 700
musk.mat: (3062, 166), n_an: 97
optdigits.mat: (5216, 64), n_an: 150
pendigits.mat: (6870, 16), n_an: 156
pima.mat: (768, 8), n_an: 268
satellite.mat: (6435, 36), n_an: 2036
satimage-2.mat: (5803, 36), n_an: 71
shuttle.mat: (49097, 9), n_an: 3511
vertebral.mat: (240, 6), n_an: 30
vowels.mat: (1456, 12), n_an: 50
wbc.mat: (378, 30), n_an: 21

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# #
# #
# #

# aor_scores_train = np.load('/kaggle/working/ranking_arrhythmia.mat_train_ob20_normalized_20000_0.npy')
# auc_train = np.load('/kaggle/working/auc_curve_train_arrhythmia.mat_20000_0.npy')
# auc_test = np.load('/kaggle/working/auc_curve_arrhythmia.mat_20000_0.npy')
# V_auc = np.load('/kaggle/working/q_auc_arrhythmia.mat_20000_0.npy')
# aor_count_train = np.load('/kaggle/working/count_arrhythmia.mat_train_ob20_normalized_20000_0.npy')
# ss_s = np.load('/kaggle/working/subspace_scores_arrhythmia.mat_20000_0.npy')

# #
# #
# #
# d = len(aor_scores_train)
# pos = np.arange(d)

# plt.figure()
# plt.plot(pos, aor_scores_train)
# plt.grid()
# plt.xlabel('Feature ranking')
# plt.ylabel('AOR')
# plt.title('Average Of Rewards')

# #
# # Plot AUC Train
# #

# auc = auc_train

# a_dim = np.argmax(auc)
# a_auc = auc[a_dim]
# a_dim = a_dim+1

# alpha0 = 0.5
# alpha = alpha0 / d
# print(f'alpha = {alpha}')
# quality = auc - alpha * pos

# q_dim = np.argmax(quality)
# q_auc = auc[q_dim]
# q_dim = q_dim+1

# ss_dim = np.argmax(ss_s)

# plt.figure()
# plt.axis([0, d, np.min(auc), np.max(auc)*1.025])
# plt.plot(pos, auc, color='b', linestyle='-', label = f'AUC ({a_dim},{a_auc:.3f})')
# plt.plot(pos, quality, color='r', linestyle='--', label = f'Quality ({q_dim},{q_auc:.3f})')
# plt.plot(pos, V_auc, color='g', linestyle='-', label = 'AUC_V')
# plt.plot(q_dim-1,q_auc,'ro', a_dim-1, a_auc,'b^')
# plt.legend(loc = 'lower right')
# plt.grid()
# plt.xlabel('Feature ranking')
# plt.title('AUC & Quality: Training set')

# print(f'Best according to AUC: dim = {a_dim} ({a_dim/d*100:.2f}%), AUC = {a_auc:.5f}')
# print(f'Best according to Quality: dim = {q_dim} ({q_dim/d*100:.2f}%), AUC = {q_auc:.5f}')

# #
# #
# #

# auc = auc_test

# a_dim = np.argmax(auc)
# a_auc = auc[a_dim]
# a_dim = a_dim+1

# alpha0 = 0.5
# alpha = alpha0 / d
# print(f'alpha = {alpha}')
# quality = auc - alpha * pos

# q_dim = np.argmax(quality)
# q_auc = auc[q_dim]
# q_dim = q_dim+1

# plt.figure()
# plt.axis([0, d, np.min(auc), np.max(auc)*1.025])
# plt.plot(pos, auc, color='b', linestyle='-', label = f'AUC ({a_dim},{a_auc:.3f})')
# plt.plot(pos, quality, color='r', linestyle='--', label = f'Quality ({q_dim},{q_auc:.3f})')
# plt.plot(pos, V_auc, color='g', linestyle='-', label = 'AUC_V')
# plt.plot(q_dim-1,q_auc,'ro', a_dim-1, a_auc,'b^')
# plt.legend(loc = 'lower right')
# plt.grid()
# plt.xlabel('Feature ranking')
# plt.title('AUC & Quality: Test set')

# print(f'Best according to AUC: dim = {a_dim} ({a_dim/d*100:.2f}%), AUC = {a_auc:.5f}')
# print(f'Best according to Quality: dim = {q_dim} ({q_dim/d*100:.2f}%), AUC = {q_auc:.5f}')

# #
# #
# #

# plt.figure()
# plt.plot(pos, aor_count_train)
# plt.xlabel('Feature ranking')
# #plt.ylabel('Feature ranking')
# plt.title('Feature count')


# #plt.figure()
# #plt.plot(pos, V_f)
# #plt.title('Value Function')
# #plt.xlabel('Feature ranking')
# ##plt.ylabel('')
# #plt.grid()

# plt.figure()
# plt.plot(pos, ss_s, label = f'AUC ({ss_dim},{ss_s[ss_dim]:.3f})')
# plt.title('Subspace score')
# plt.xlabel('Feature ranking')
# #plt.ylabel('')
# plt.grid()
# plt.legend(loc = 'lower right')



# #mean(Q_f)

In [ ]:
# print([f"{aucsd:.3f}" for aucsd in all_aucs])
# print(np.max(all_aucs))
# print(np.argmax(all_aucs))

# print(ss_s)
# print(aor_scores_train)
# ids = np.argsort(ranking_train)
# print(ranking_train)
# print(f"INFORMATIVE FEATURES: {ranking_train[ids][important_mask==1]}")
# print(f"INFORMATIVE FEATURE AOR: {aor_scores_train[ids][important_mask==1]}")

In [ ]:
# import os

# def aggregate_results(datasets, steps, runs, base_dir="dir"):
    
#     result_dict = {}

#     for dataset in datasets:
#         all_auc = []
#         all_auc_train = []
#         all_aor = []
#         all_count = []
#         all_rankings = []

#         for r in range(runs):
#             auc_path = f"auc_curve_{dataset}_{steps}_{r}.npy"
#             auc_train_path = f"auc_curve_train_{dataset}_{steps}_{r}.npy"
#             aor_path = f"ranking_{dataset}_train_ob20_normalized_{steps}_{r}.npy"
#             count_path = f"count_{dataset}_train_ob20_normalized_{steps}_{r}.npy"
#             rank_path = f"ranked_features_{dataset}_train_ob20_normalized_{steps}_{r}.npy"

#             all_auc.append(np.load(auc_path))
#             all_auc_train.append(np.load(auc_train_path))
#             all_aor.append(np.load(aor_path))
#             all_count.append(np.load(count_path))
#             all_rankings.append(np.load(rank_path))

        
#         all_auc = np.array(all_auc)
#         all_auc_train = np.array(all_auc_train)
#         all_aor = np.array(all_aor)
#         all_count = np.array(all_count)
#         all_rankings = np.array(all_rankings)

#         mean_auc = np.mean(all_auc, axis=0)
#         std_auc = np.std(all_auc, axis=0)

#         mean_auc_train = np.mean(all_auc_train, axis=0)
#         std_auc_train = np.std(all_auc_train, axis=0)

#         mean_aor = np.mean(all_aor, axis=0)
#         std_aor = np.std(all_aor, axis=0)

#         mean_count = np.mean(all_count, axis=0)
#         std_count = np.std(all_count, axis=0)

#         ranking_scores = np.zeros_like(all_rankings[0], dtype=np.float32)
#         for r in range(all_rankings.shape[0]):
#             ranking_scores += np.argsort(np.argsort(all_rankings[r]))
#         ranking_scores /= all_rankings.shape[0]
#         mean_ranking = np.argsort(ranking_scores)

#         result_dict[dataset] = {
#             "mean_auc": mean_auc,
#             "std_auc": std_auc,
#             "mean_auc_train": mean_auc_train,
#             "std_auc_train": std_auc_train,
#             "mean_aor": mean_aor,
#             "std_aor": std_aor,
#             "mean_count": mean_count,
#             "std_count": std_count,
#             "mean_ranking": mean_ranking,
#             "rank_stability": ranking_scores
#         }

#     return result_dict

In [ ]:
#import shutil
#shutil.make_archive('/kaggle/working/output', 'zip', '/kaggle/working')